# Plotter 3.0

In [73]:
import base64
import os, io
from datetime import date, datetime, timedelta
import pandas as pd
import seaborn as sns
from windrose import WindroseAxes
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from PIL import Image
import numpy as np
import google.cloud.logging
from google.cloud import storage
import logging
from alive_progress import alive_bar

In [74]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../service_account.json'

### Create Date Range 

In [75]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----- ----- ----- -----    CREATE DATERANGE    ----- ----- ----- -----
# Create list of all dates between start_date and end_date
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# ======================================================================

### Truncate CMAP

In [76]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----- ----- ----- -----   TRUNCATE COLORMAP    ----- ----- ----- -----
# Can be used to remove white / black bookended values
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
          'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
          cmap(np.linspace(minval, maxval, n)))
    return new_cmap

# ======================================================================

### Import Weather Data

In [77]:
# -------------------------------------------------------------------------------
# IMPORT DATASET
# Function to import all weather data
# Data Patching completed in this step
# -------------------------------------------------------------------------------
def import_weather_data(storage_client):
    # Connect to Google Cloud Storage
    # -------------------------------
    storage_client = storage.Client()

    # Open Bucket
    # -----------
    bucket_name = 'weather_aurorabc'
    bucket = storage_client.bucket(bucket_name)

    # Open Blob
    # ---------
    blob_name = 'weather_ampm.csv'
    #blob_name = 'backups/weather-2023-09-26.csv'
    blob = bucket.blob(blob_name)

    # Read Weather Data from Blob
    # ---------------------------
    with blob.open("r") as f:
        weather_data = pd.read_csv(f)

    # Cast all data variables to float
    weather_data = weather_data.astype(
        {'Height':'float',
        'Temp':'float',
        'Dew_Point':'float',
        'Relative_Humidity':'float',
        'Mixing_Ratio':'float',
        'Wind_Direction':'float',
        'Wind_Speed':'float',
        'Potential_Temp':'float',
        'Equivalent_Potential_Temp':'float',
        'Virtual_Potential_Temp':'float',
        })

    return weather_data

# Find Missing Plots
---

In [78]:
# -------------------------------------------------------------------------------
# FIND MISSING PLOTS
# Function to find any dates with missing plots
# Returns a list of dates to be plot
# -------------------------------------------------------------------------------
def find_missing_plots(storage_client, locations, plot_range):
    
    # Create Date Range
    # ~~~~~~~~~~~~~~~~~
    start_date = datetime(2023, 8, 20)
    end_date = datetime.today()
    search_range = []
    for single_date in daterange(start_date, end_date):
        search_range.append(single_date.date())
                
    # Define static params
    # ~~~~~~~~~~~~~~~~~~~~
    bucket_name = 'weather_aurorabc'
    bucket = storage_client.bucket(bucket_name)
    plots = ['temp', 'wind']
    
        
    # Search through range of dates for missing graphs
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    missing_dates = []
    for date in search_range:
        for loc in locations:
            for plot in plots:
                for r in plot_range:
                    blob_name = 'graphs/{}/{}_{}_{}d_{}.webp'.format(date.strftime("%Y-%m-%d"), loc, plot, r, date.strftime("%Y-%m-%d"))
                    #print("Checking for file...\n{}\n".format(blob_name))
                    found = storage.Blob(bucket=bucket, name=blob_name).exists(storage_client)
                    #print("Status: {}".format(status))
                    if not found:
                        print(date)
                        missing_dates.append(date)
                        break
                if date in missing_dates:
                    break
            if date in missing_dates:
                break

        
    return missing_dates

# Plot Temps
---
Updated function to plot temperature graphs of varying data ranges for a range of dates for individual locations

In [79]:
def plot_temps(storage_client, location, dates, plot_range):

    for d in dates:
        for r in plot_range:
            
            # Set figure details
            # ~~~~~~~~~~~~~~~~~~
            plt.figure(figsize=(20,6))
            plt.xlim([0, r -1])
            plt.grid()
            plt.axhline(0, color='black')
            plt.xticks(rotation = 'vertical')
            plt.title("Temperature °C - {} day".format(r))
            if r > 30:
                plt.xticks(np.arange(0, r+1, 7.0))

            # Set color palette for the graph
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            palette = sns.color_palette("Paired", n_colors=6)
            palette_order = [1, 0, 3, 2, 5, 4]
            palette = [palette[idx] for idx in palette_order]
            hue_order = [850, 700]


            # Create new df containing only data within plot range
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            mask = (location['data']['Date'] >= str(d - timedelta(days=r))) & (location['data']['Date'] <= str(d))
            plot_data = location['data'].loc[mask]
            
            # Calculate IQR for column Temp
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            Q1 = plot_data['Temp'].quantile(0.10)
            Q3 = plot_data['Temp'].quantile(0.90)
            IQR = Q3 - Q1

            # Identify and remove outliers
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            threshold = 1.5
            outliers = plot_data[(plot_data['Temp'] < Q1 - threshold * IQR) | (plot_data['Temp'] > Q3 + threshold * IQR)]
            plot_data = plot_data.drop(outliers.index)
            
            # Skip plot if there are no numerical values within the subset
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if plot_data.dropna(subset=['Temp']).empty:
                print("COULD NOT GENERATE GRAPH")
                print("Location: {}".format(location['name']))
                print("Date: {}".format(d))
                print("Plot Range: {}".format(r))
                print("Data: {}\n".format(plot_data))
                continue

            # Plot graph
            # ~~~~~~~~~~
            try:
                g = sns.lineplot(x='Date', y='Temp', hue='Pressure', data=plot_data, palette=palette[location['palette_order'][0]: location['palette_order'][1]], hue_order=hue_order)
            except Exception as e:
                print(e)
                print(plot_data)

            # Rename legend with detailed labels
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            l = plt.legend()
            labels = ['850', '700']
            for x, label in enumerate(labels):
                l.get_texts()[x].set_text(label)
                
            # Define save details
            # ~~~~~~~~~~~~~~~~~~~
            save_name = '{}_temp'.format(location['save_name'])
            save_loc = "./figures/temp/{}".format(d.strftime("%Y-%m-%d"))
            if not os.path.isdir(save_loc):
                os.makedirs(save_loc)
                
            # Publish graph to Google Cloud
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            bucket_name = 'weather_aurorabc'
            bucket = storage_client.bucket(bucket_name)

            buff = io.BytesIO()
            plt.savefig(buff, format='webp',dpi=150, bbox_inches="tight")

            blob_name = 'graphs/{}/{}_{}d_{}.webp'.format(d.strftime("%Y-%m-%d"), save_name, r, d.strftime("%Y-%m-%d"))
            blob = bucket.blob(blob_name)
            blob.upload_from_string(buff.getvalue(), content_type='image/png')
            plt.close()
            del buff

# Plot ALL Temps
---
Updated function to plot temperature graphs of varying data ranges for a range of dates for all locations on a single graph

In [80]:
def plot_all_temps(storage_client, locations, dates, plot_range):
    
    for d in dates:
        for r in plot_range:
            
            # Set figure details
            # ~~~~~~~~~~~~~~~~~~
            plt.figure(figsize=(20,6))
            plt.xlim([0, r -1])
            plt.grid()
            plt.axhline(0, color='black')
            plt.xticks(rotation = 'vertical')
            plt.title("Temperature °C - {} day".format(r))
            if r > 30:
                plt.xticks(np.arange(0, r+1, 7.0))
                
            # Set color palette for the graph
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            palette = sns.color_palette("Paired", n_colors=6)
            palette_order = [1, 0, 3, 2, 5, 4]
            palette = [palette[idx] for idx in palette_order]
            hue_order = [850, 700]
            
            skip = False
            
            # Loop through each location and plot data on graph
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            for location in locations:
                if skip:
                    continue
                # Create truncated data object with only correct date range
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                mask = (location['data']['Date'] >= str(d - timedelta(days=r))) & (location['data']['Date'] <= str(d))
                plot_data = location['data'].loc[mask]
                
                # Calculate IQR for column Temp
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                Q1 = plot_data['Temp'].quantile(0.10)
                Q3 = plot_data['Temp'].quantile(0.90)
                IQR = Q3 - Q1
                
                # Identify and remove outliers
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                threshold = 1.5
                outliers = plot_data[(plot_data['Temp'] < Q1 - threshold * IQR) | (plot_data['Temp'] > Q3 + threshold * IQR)]
                plot_data = plot_data.drop(outliers.index)
                
                # Skip plot if there are no numerical values within the subset
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                if plot_data.dropna(subset=['Temp']).empty:
                    print("COULD NOT GENERATE GRAPH")
                    print("Location: {}".format(location['name']))
                    print("Date: {}".format(d))
                    print("Plot Range: {}".format(r))
                    print("Data: {}\n".format(plot_data))
                    skip = True
                    continue
                

                # Add data for single location to graph
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                g = sns.lineplot(x='Date', y='Temp', hue='Pressure', data=plot_data, palette=palette[location['palette_order'][0]: location['palette_order'][1]], hue_order=hue_order)
                
            
            if not skip:
                # Rename legend with detailed labels
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                l = plt.legend()
                labels = []
                for location in locations:
                    for pressure in [700, 850]:
                        labels.append('{} {}'.format(location['name'], pressure))

                labels = [labels[idx] for idx in palette_order]
                for x, label in enumerate(labels):
                    l.get_texts()[x].set_text(label)


                # Generate Save Path
                # ~~~~~~~~~~~~~~~~~~
                save_name = 'all_temp'
                save_loc = "./figures/wind/{}".format(d)
                if not os.path.isdir(save_loc):
                    os.makedirs(save_loc)


                # Publish graph to Google Cloud
                # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                bucket_name = 'weather_aurorabc'
                bucket = storage_client.bucket(bucket_name)

                buff = io.BytesIO()
                plt.savefig(buff, format='webp',dpi=150, bbox_inches="tight")

                blob_name = 'graphs/{}/{}_{}d_{}.webp'.format(d.strftime("%Y-%m-%d"), save_name, r, d.strftime("%Y-%m-%d"))
                blob = bucket.blob(blob_name)
                blob.upload_from_string(buff.getvalue(), content_type='image/png')
                plt.close()
                del buff

# Plot Wind
---
Updated function to plot wind graphs of varying data ranges for a range of dates for individual locations

In [81]:
def plot_wind(storage_client, location, dates, plot_range):
    
    # Define color map and truncate to apropriate range
    cmap = cm.viridis_r
    cmap = truncate_colormap(cmap, minval=0, maxval=0.9)
    
    for d in dates:
        for r in plot_range:
            
            # Create new df containing only data within plot range
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            mask = (location['data']['Date'] >= str(d - timedelta(days=r))) & (location['data']['Date'] <= str(d))
            plot_data = location['data'].loc[mask]
            
            # Calculate IQR for column Temp
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            Q1 = plot_data['Wind_Speed'].quantile(0.05)
            Q3 = plot_data['Wind_Speed'].quantile(0.95)
            IQR = Q3 - Q1

            # Identify and remove outliers
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            threshold = 1.5
            outliers = plot_data[(plot_data['Wind_Speed'] < Q1 - threshold * IQR) | (plot_data['Wind_Speed'] > Q3 + threshold * IQR)]
            plot_data = plot_data.drop(outliers.index)
            plot_data = plot_data.dropna(subset=['Wind_Speed'])
            
            # Skip plot if there are no numerical values within the subset
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if plot_data.empty:
                print("COULD NOT GENERATE GRAPH")
                print("Location: {}".format(location['name']))
                print("Date: {}".format(d))
                print("Plot Range: {}".format(r))
                print("Data: {}\n".format(plot_data))
                continue
                
            # Populate speed and direction vars from DataFrame
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
            wd = plot_data['Wind_Direction'].to_list()
            ws = plot_data['Wind_Speed'].to_list()


            # Plot wind rose for data set
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~
            ax = WindroseAxes.from_ax()
            #ax.contourf(wd, ws, bins=np.arange(0, 100, 10), cmap=cmap)
            ax.contourf(wd, ws, cmap=cmap)
            ax.set_legend()
            
            # Save figure
            # ~~~~~~~~~~~
            plt.title("{} Wind - {} day".format(location['name'], r))
            plt.legend(title="Wind Speed (km/h)", loc=4, fontsize='small', fancybox=True)
            
            # Generate Save Path
            # ~~~~~~~~~~~~~~~~~~
            save_name = '{}_wind'.format(location['name'].lower().replace(' ', ''))
            save_loc = "./figures/wind/{}".format(d)
            
            if not os.path.isdir(save_loc):
                os.makedirs(save_loc)

            # Save image as webp
            # ~~~~~~~~~~~~~~~~~~
            path = '{}/{}_{}d_{}'.format(save_loc, save_name, r, d)
            plt.savefig('{}.webp'.format(path), format='webp',dpi=150)
            
            # Publish graph to Google Cloud
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            bucket_name = 'weather_aurorabc'
            bucket = storage_client.bucket(bucket_name)

            buff = io.BytesIO()
            plt.savefig(buff, format='webp',dpi=150, bbox_inches="tight")

            blob_name = 'graphs/{}/{}_{}d_{}.webp'.format(d, save_name, r, d)
            blob = bucket.blob(blob_name)
            blob.upload_from_string(buff.getvalue(), content_type='image/png')
            plt.close()
            del buff

### Main Application Body

In [82]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# ----- ----- ----- -----          START         ----- ----- ----- -----
#  + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def main():
    # Collect weather data
    # ~~~~~~~~~~~~~~~~~~~~
    storage_client = storage.Client()
    weather_data = import_weather_data(storage_client)        
    
    # Define location specific params
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    locations = [{
            'name': "Vernon",
            'palette_order': [2, 4], # green
            'save_name': "vernon",
            'data': weather_data[weather_data['Station'] == "Vernon"]
        },
        {
            'name': "Port Hardy",
            'palette_order': [4, 6], # red
            'save_name': "porthardy",
            'data': weather_data[weather_data['Station'] == "Port Hardy"]
        },
        {
            'name': "Quillayute",
            'palette_order': [0, 2], # blue
            'save_name': "quillayute",
            'data': weather_data[weather_data['Station'] == "Quillayute"]
        }]
    
    # Define plot ranges
    # ~~~~~~~~~~~~~~~~~~
    plot_range = [3, 5, 10, 30, 90, 180]
    
    # Generate list of dates to plot graphs for
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print("Scanning for missing graphs...")
    dates = find_missing_plots(storage_client, [loc['save_name'] for loc in locations], plot_range)
    
    with alive_bar(len(dates), force_tty=True) as bar:
        for date in dates:
            print('{} --------------------'.format(date))
            print("Plotting ALL Temps")
            plot_all_temps(storage_client, locations, [date], plot_range)

            for location in locations:

                print("Plotting Wind: {}".format(location['name']))
                plot_wind(storage_client, location, [date], plot_range)

                print("Plotting Temp: {}\n".format(location['name']))
                plot_temps(storage_client, location, [date], plot_range)
            bar()
        
    
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# ----- ----- ----- -----           END          ----- ----- ----- -----
#  + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [83]:
main()

Scanning for missing graphs...
|████████████████████████████████████████| 0 in 0.1s (0.00/s)                                                           


In [ ]:
# Create function to find missing graphs --- DONE
# Create function to plot all temps together --- DONE
# Create function to plot wind --- DONE

In [ ]:
# Plotting Temps Error
# missing data is removed and therefore so are x axis entries
# can this wait until other issues are resolved?